<center>
    <h1> Programación Científica </h1>
    <h1> Actividad Clase 6 </h1>
    <h3> Aldo Berrios Valenzuela  </h3>
    <h3> 201304018-7 </h3>
    <h4> `aldo.berrios.13@sansano.usm.cl` </h4>
</center>

# 1. &nbsp;&nbsp;&nbsp;&nbsp; Preliminares

Importamos las librerías y funciones que usaremos durante la experiencia:

In [4]:
import numba
import numexpr as ne
import numpy as np
import matplotlib.pyplot as roberplot
import matplotlib.image as mpimg

%load_ext memory_profiler

def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

def image_plot(img):
    roberplot.figure(figsize=(10,5))
    roberplot.imshow(img, cmap='gray')
    roberplot.axis('off')
    roberplot.show()
    
def histogram_plot(h):
    roberplot.figure(figsize=(5,10))
    roberplot.plot(range(len(h)), h)
    roberplot.xlim([-5,260])
    roberplot.ylim([0,200000])
    roberplot.xlabel('Feature index')
    roberplot.ylabel('Number of repetitions')
    roberplot.grid()
    roberplot.show()

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


y las funciones relacionadas con el *Local Binary Pattern Representation*:

In [2]:
def lbp(region):
    """
    Region is a 3x3 ndarray
    """
    ret = (region > region[1,1]).astype(int)
    return (2**7)*ret[0,0] + (2**6)*ret[0,1] + (2**5)*ret[0,2] + (2**4)*ret[1,2] + (2**3)*ret[2,2] + \
           (2**2)*ret[2,1] + (2**1)*ret[2,0] + (2**0)*ret[1,0]


def lbp_representation(image):
    """
    Compute lbp representation of image
    """
    m,n = image.shape
    ret = np.empty((m-2,n-2))
    for i in range(1,m-1):
        for j in range(1,n-1):
            ret[i-1,j-1] = lbp(image[i-1:i+2, j-1:j+2])
    return ret


def lbp_histogram(image):
    """
    Compute lbp histogram representation of image
    """
    lbp_image = lbp_representation(image)
    m,n = lbp_image.shape
    hist = np.zeros(256)
    for i in range(m):
        for j in range(n):
            hist[lbp_image[i,j]] += 1
    return hist 

# 2. &nbsp;&nbsp;&nbsp;&nbsp; Preguntas

## 2.1. &nbsp;&nbsp;&nbsp; Detección de Cuellos de Botella

Para encontrar los cuellos de botella en `lbp_histogram(image)` debemos hacer un line profiling y así, buscar la línea que más demora en ejecutarse para posteriormente hacer los cambios respectivos. Para ello, comenzaremos importando el módulo `line_profiler`:

In [8]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [9]:
img = mpimg.imread('doge.jpg')
img = rgb2gray(img)

In [11]:
%lprun -T lbp_histogram_lprof -f lbp_histogram lbp_histogram(img)

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:30: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future



*** Profile printout saved to text file 'lbp_histogram_lprof'. 


Luego, procedemos a leer los resultados del line profiling anterior:

In [12]:
print(open('lbp_histogram_lprof', 'r').read())

Timer unit: 1e-06 s

Total time: 50.6859 s
File: <ipython-input-2-e53e5d40ad48>
Function: lbp_histogram at line 21

Line #      Hits         Time  Per Hit   % Time  Line Contents
    21                                           def lbp_histogram(image):
    22                                               """
    23                                               Compute lbp histogram representation of image
    24                                               """
    25         1     34782100 34782100.0     68.6      lbp_image = lbp_representation(image)
    26         1            3      3.0      0.0      m,n = lbp_image.shape
    27         1           10     10.0      0.0      hist = np.zeros(256)
    28      1465          670      0.5      0.0      for i in range(m):
    29   3576552      1782422      0.5      3.5          for j in range(n):
    30   3575088     14120691      3.9     27.9              hist[lbp_image[i,j]] += 1
    31         1            1      1.0      0.0      ret

Como podemos observar, la función `lbp_histogram(image)` se demora bastante en ejecutar las lineas `lbp_image = lbp_representation(image)` y `hist[lbp_image[i,j]] += 1` ya se que llevan el mayor porcentaje en tiempos de ejecución del algoritmo.

## 2.2. &nbsp;&nbsp;&nbsp; Modificación de Funciones para solucionar cuellos de botella

Entre las primeras cosas que haremos para solucionar los cuellos de botella de `lbp_histogram(image)` será cambiar las líneas:
```python
for i in range(m):
    for j in range(n):
        hist[lbp_image[i,j]] += 1
```
que lo único que hacen es contar cuantas veces se repite un número del 0 al 255 en `lbp_image`. Resulta mucho más conveniente iterar desde el 0 al 255 y usar funciones de numpy que nos diga cuantas veces se repite este número. Para ello, usaremos `np.count_nonzero`. En consiguiente reemplazamos el doble `for` mostrado anteriormente por:
```python
for i in range(256):
    hist[i] = np.count_nonzero(lbp_image == i)
```


In [23]:
def lbp_histogram_new(image):
    """
    Compute lbp histogram representation of image
    """
    lbp_image = lbp_representation(image)
    m,n = lbp_image.shape
    hist = np.zeros(256)
    for i in range(256):
        hist[i] = np.count_nonzero(lbp_image == i)
    return hist

Por otro lado, 

In [22]:
%lprun -T lbp_histogram_lprof_new -f lbp_histogram_new lbp_histogram_new(img)


*** Profile printout saved to text file 'lbp_histogram_lprof'. 
